In [1]:
import math
import time
import os
import gc
import sys
import logging
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import Dataset
import torch.nn.functional as F 
from torch.utils.data import DataLoader

In [2]:
sys.path.append("./utils/")
from nn_utils import EarlyStopping
from nn_scheduler import NullScheduler, adjust_learning_rate, get_learning_rate

## Load Data

In [3]:
df = pd.read_pickle("../data/train_data/train.pkl")

In [4]:
df.head()

,bakaze,kyoku_num,honba,kyotaku,dora,parent,riichi_player,agari_hai,player0_tehai,player0_sutehai,player0_point,player1_tehai,player1_sutehai,player1_point,player2_tehai,player2_sutehai,player2_point,player3_tehai,player3_sutehai,player3_point
0,0,1,0,0,[24],0,0,"[12, 15]","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250
1,0,1,1,0,[26],0,0,[9],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308
2,0,1,2,0,[31],0,2,[7],"[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, ...","[34, 31, 36, 35, 33, 1, 21, 29, 33, 11, 13, 19...",171,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...","[21, 37, 9, 37, 32, 31, 28, 3, 9, 33, 32, 8, 16]",381,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, ...","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, ...","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219
3,0,2,0,0,[32],1,3,[28],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[34, 21, 22, 12, 35, 12, 4, 15]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197
4,0,3,0,0,[1],2,1,"[1, 4]","[0, 0, 1, 1, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 0, ...","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[0, 0, 1, 1, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 0, ...","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117


In [5]:
def func_33_to_40(x):
    if x < 9:
        return x+1
    elif x < 18:
        return x+2
    elif x < 27:
        return x+3
    else:
        return x+4
df["dora"] = df["dora"].map(lambda h_list: [func_33_to_40(h//4) for h in h_list])

In [6]:
def preprocess(df):
    """
    0. doraカラムの修正
    1.使わないカラムを事前に消しておく
        Transformer任せなので多めに削る
    2. リーチ者が何家か
    3. リーチ者が親かどうか
    """
    def func_33_to_40(x):
        if x < 9:
            return x+1
        elif x < 18:
            return x+2
        elif x < 27:
            return x+3
        else:
            return x+4
    df["dora"] = df["dora"].map(lambda h_list: [func_33_to_40(h//4) for h in h_list])
    df["is_parent"] = (df["parent"] == df["riichi_player"]).astype(int)
    df["riichi_kaze"] = (df["riichi_player"] - df["kyoku_num"])%4
    drop_cols = [
        "bakaze",
        "kyoku_num",
        "honba",
        "kyotaku",
        "parent",
        "riichi_player",
        "player0_point",
        "player1_point",
        "player2_point",
        "player3_point",
        "player0_tehai",
        "player2_tehai",
        "player3_tehai",
    ]
    return df.drop(drop_cols, axis=1)

In [7]:
df = preprocess(df)

In [36]:
df.head()

,dora,agari_hai,player0_sutehai,player1_tehai,player1_sutehai,player2_sutehai,player3_sutehai,is_parent,riichi_kaze
0,[2],"[12, 15]","[9, 1, 11, 16, 5]","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]","[9, 1, 11, 16, 5]","[9, 1, 11, 16, 5]",1,3
1,[2],[9],"[33, 19, 32, 34, 35, 12, 4, 14, 7]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]","[33, 19, 32, 34, 35, 12, 4, 14, 7]","[33, 19, 32, 34, 35, 12, 4, 14, 7]",1,3
2,[3],[7],"[34, 31, 36, 35, 33, 1, 21, 29, 33, 11, 13, 19...","[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...","[21, 37, 9, 37, 32, 31, 28, 3, 9, 33, 32, 8, 16]","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",0,1
3,[3],[28],"[34, 21, 22, 12, 35, 12, 4, 15]","[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]","[29, 34, 37, 35, 36, 9, 33]","[29, 34, 37, 35, 36, 9, 33]",0,1
4,[1],"[1, 4]","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]","[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",0,2


In [9]:
def make_hai_count(df):
    # 各牌が場に何枚見えているか
    # 各プレイヤーの捨て牌
    print("player0...")
    sutehai_count0 = df["player0_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    print("player1...")
    sutehai_count1 = df["player1_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    print("player2...")
    sutehai_count2 = df["player2_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    print("player3...")
    sutehai_count3 = df["player3_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    sutehai_count_all = sutehai_count0 + sutehai_count1 + sutehai_count2 + sutehai_count3
    # プレイヤー1目線として、プレイヤー1の手牌
    print("player1 tehai...")
    tehai_count1 = df["player1_tehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    # ドラ表示牌
    def pre_dora_func(x):
        if x < 30:
            if x%10 > 1:
                return x-1
            else:
                return x//10 + 9
        else:
            if x == 31:
                return 34
            if x == 35:
                return 37
            else:
                return x-1                
    print("dora...")
    pre_dora = df["dora"].map(
        lambda h_list: [pre_dora_func(h) for h in h_list]).apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    # sum
    count_all = sutehai_count_all + tehai_count1
    use_col = [c for c in range(40) if c < 38 and c%10 != 0]
    count_all = count_all.loc[:, use_col]
    count_all.columns = [f"hai_count_{c}" for c in use_col]
    print("complete!!!")
    return count_all

In [10]:
hai_counts = make_hai_count(df)

player0...
player1...
player2...
player3...
player1 tehai...
dora...
complete!!!


In [11]:
df.head()

,dora,agari_hai,player0_sutehai,player1_tehai,player1_sutehai,player2_sutehai,player3_sutehai,is_parent,riichi_kaze
0,[2],"[12, 15]","[9, 1, 11, 16, 5]","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]","[9, 1, 11, 16, 5]","[9, 1, 11, 16, 5]",1,3
1,[2],[9],"[33, 19, 32, 34, 35, 12, 4, 14, 7]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]","[33, 19, 32, 34, 35, 12, 4, 14, 7]","[33, 19, 32, 34, 35, 12, 4, 14, 7]",1,3
2,[3],[7],"[34, 31, 36, 35, 33, 1, 21, 29, 33, 11, 13, 19...","[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...","[21, 37, 9, 37, 32, 31, 28, 3, 9, 33, 32, 8, 16]","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",0,1
3,[3],[28],"[34, 21, 22, 12, 35, 12, 4, 15]","[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]","[29, 34, 37, 35, 36, 9, 33]","[29, 34, 37, 35, 36, 9, 33]",0,1
4,[1],"[1, 4]","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]","[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",0,2


In [12]:
hai_counts.head()

,hai_count_1,hai_count_2,hai_count_3,hai_count_4,hai_count_5,hai_count_6,hai_count_7,hai_count_8,hai_count_9,hai_count_11,...,hai_count_27,hai_count_28,hai_count_29,hai_count_31,hai_count_32,hai_count_33,hai_count_34,hai_count_35,hai_count_36,hai_count_37
0,14.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.0,0.0,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0
2,16.0,3.0,1.0,0.0,2.0,4.0,2.0,1.0,2.0,1.0,...,2.0,3.0,3.0,2.0,2.0,5.0,1.0,1.0,1.0,4.0
3,9.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,3.0,0.0,0.0,3.0,4.0,4.0,3.0,3.0
4,5.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,4.0,0.0,4.0,4.0,2.0,2.0,4.0,0.0


In [13]:
df.to_pickle("002_train_df.pkl")

In [14]:
hai_counts.to_pickle("002_hai_counts.pkl")

## Dataset

In [28]:
class MJDataset(Dataset):
    def __init__(self, df, hai_counts):
        self.agari_hai = df["agari_hai"].values
        self.riichi_sutehai = df["player0_sutehai"].values
        self.hai_counts = hai_counts.values

    def __len__(self):
        return len(self.agari_hai_arr)

    def __getitem__(self, idx):
        riichi_sutehai = np.array(self.riichi_sutehai[idx]).astype(np.int32)
        hai_count = np.array(self.hai_counts[idx]).astype(np.int32)
        agari_hai = np.array(self.agari_hai[idx]).astype(np.float32)
        return riichi_sutehai, hai_count, agari_hai

## Define Archtecture

In [16]:
class Attention(nn.Module):
    '''
    Scaled Dot Production
    Multi-head attention
    '''

    def __init__(self, d_model=300, n_head=1):
        super().__init__()

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)

        self.out = nn.Linear(d_model, d_model)
        # multi-head attention head num
        self.n_head = n_head
        # Attentionの大きさ調整の変数
        self.d_k = d_model

    def _reshape_to_batches(self, x):
        batch_size, seq_len, in_feature = x.size()
        sub_dim = in_feature // self.n_head
        return x.reshape(batch_size, seq_len, self.n_head, sub_dim)\
                .permute(0, 2, 1, 3)\
                .reshape(batch_size * self.n_head, seq_len, sub_dim)

    def _reshape_from_batches(self, x):
        batch_size, seq_len, in_feature = x.size()
        batch_size //= self.n_head
        out_dim = in_feature * self.n_head
        return x.reshape(batch_size, self.n_head, seq_len, in_feature)\
                .permute(0, 2, 1, 3)\
                .reshape(batch_size, seq_len, out_dim)

    def forward(self, q, k, v, mask):

        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)

        # multihead
        if self.n_head > 1:
            q = self._reshape_to_batches(q)
            k = self._reshape_to_batches(k)
            v = self._reshape_to_batches(v)

        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)

        # ここでmaskを計算
        mask = mask.unsqueeze(1)
        if mask is not None and self.n_head > 1:
            mask = mask.repeat(self.n_head, 1, 1)
        weights = weights.masked_fill(mask == 0, -1e9)

        # softmaxで規格化をする
        normlized_weights = F.softmax(weights, dim=-1)

        # AttentionをValueとかけ算
        output = torch.matmul(normlized_weights, v)
        
        if self.n_head > 1:
            output = self._reshape_from_batches(output)

        # 全結合層で特徴量を変換
        output = self.out(output)

        return output, normlized_weights

In [17]:
class LinearRn(nn.Module):
    def __init__(self, in_channel, out_channel, dropout=0.3):
        super(LinearBn, self).__init__()
        self.linear = nn.Linear(in_channel, out_channel)
        self.ln = nn.LayerNorm(out_channel)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = self.linear(x)
        x = self.ln(x)
        x = self.dropout(F.relu(x))
        return x

In [18]:
class FeedForward(nn.Module):
    def __init__(self, n_input, n_output, n_hidden=256, dropout=0.3):
        super().__init__()

        self.linear_1 = nn.Linear(n_input, n_hidden)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x

In [19]:
class PositionalEncoder(nn.Module):

    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()
        self.d_model = d_model 
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * (i + 1))/d_model)))
        self.pe = pe.unsqueeze(0)
        self.pe.requires_grad = False

    def forward(self, x):
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret

In [20]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1, n_head=1):
        super().__init__()
        # multi-head num
        self.n_head = n_head
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        # Attention Layer
        self.attn = Attention(d_model, n_head=n_head)
        # feed forward Layer
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        # normalize
        x_normlized = self.norm_1(x)
        # attention
        output, normlized_weights = self.attn(
            x_normlized, x_normlized, x_normlized, mask)
        
        x2 = x + self.dropout_1(output)

        x_normlized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normlized2))

        return output, normlized_weights

In [21]:
class PredictionLayer(nn.Module):
    def __init__(self, input_dim):
        super(RegressionLayer, self).__init__()
        self.linear1 = nn.Linear(input_dim, 34)

    def forward(self, x):
        h = self.linear1(x)
        h = F.sigmoid(h)
        return h

In [22]:
class MJNet(nn.Module):
    '''Transformerで麻雀の当たり牌を予測する'''

    def __init__(self, cat_nunique, n_hai_embed=8, n_head=4,
                           d_model=32, title_nunique=5, title_encode="ohe",
                           title_n_emb_out=2):
        super().__init__()
        # モデル構築
        # 牌をembedding
        self.hai_embbed = nn.Embedding(37, n_hai_embed)
        self.hai_dense = nn.Linear(n_hai_embed, d_model)
        
        # self.pe = PositionalEncoder(d_model=d_model, max_seq_len=256)
        self.tf1 = TransformerBlock(d_model=d_model, n_head=n_head)
        self.tf2 = TransformerBlock(d_model=d_model, n_head=n_head)
        self.linear_rn = LinearRn(d_model + title_nunique, 32, act=nn.ReLU(inplace=True))
        self.pred_layer = PredictionLayer(input_dim=32)

    def forward(self, x_sutehai, x_hai_count, mask, return_feature=False):
        h = self.hai_embbed(x_sutehai)
        h = self.hai_dense(h)
        # transformer
        h, normlized_weights_1 = self.tf1(h, mask)
        h, normlized_weights_2 = self.tf2(h, mask)
        h_select = h[:, 0, :] 
        # Linear層
        h = self.linear_rn(h_select)
        # 最終層
        h = self.pred_layer(h)
        return h
    
    def predict(self, x_sutehai, x_hai_count, mask):
        return self.forward(x, x_sutehai, x_hai_count, mask)
    
    def get_feature(self, x_sutehai, x_hai_count, mask):
        return self.forward(x_sutehai, x_hai_count, mask, return_feature=True)

## test

In [30]:
tmp_dataset = MJDataset(df, hai_counts)

In [31]:
tmp_dataset.__getitem__(0)

(array([ 9,  1, 11, 16,  5], dtype=int32),
 array([14,  0,  1,  0,  4,  0,  0,  0,  4,  4,  0,  0,  0,  0,  4,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       dtype=int32),
 array([12., 15.], dtype=float32))

In [32]:
hai_counts.head()

,hai_count_1,hai_count_2,hai_count_3,hai_count_4,hai_count_5,hai_count_6,hai_count_7,hai_count_8,hai_count_9,hai_count_11,...,hai_count_27,hai_count_28,hai_count_29,hai_count_31,hai_count_32,hai_count_33,hai_count_34,hai_count_35,hai_count_36,hai_count_37
0,14.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.0,0.0,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0
2,16.0,3.0,1.0,0.0,2.0,4.0,2.0,1.0,2.0,1.0,...,2.0,3.0,3.0,2.0,2.0,5.0,1.0,1.0,1.0,4.0
3,9.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,3.0,0.0,0.0,3.0,4.0,4.0,3.0,3.0
4,5.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,2.0,...,0.0,0.0,4.0,0.0,4.0,4.0,2.0,2.0,4.0,0.0
